In [16]:
%pip install requests
%pip install python-frontmatter
%pip install openai
%pip install python-dotenv
%pip install minsearch
%pip install transformers 
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached pillow-11.3.0-cp313-cp313-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (9.0 kB)
   ━━━━━━━━ 0.0/88… ? eta -:--:--
   ━━━━━━ 3.9/… 19.9   eta 0:00:…
   ━━━━━━ 8.9/… 22.1   eta 0:00:…
   ━━━━━━ 13.1… 22.3   eta 0:00:…
   ━━━━━━ 18.4… 22.7   eta 0:00:…
   ━━━━━━ 23.6… 23.5   eta 0:00:…
   ━━━━━━ 28.3… 23.6   eta 0:00:…
   ━━━━━━ 33.8… 24.1   eta 0:00:…
   ━━━━━━ 37.7… 23.6   eta 0:00:…
   ━━━━━━ 40.1… 22.2   eta 0:00:…
   ━━━━━━ 42.2… 21.0   eta 0:00:…
   ━━━━━━ 44.6… 20.1   eta 0:00:…
   ━━━━━━ 46.9… 19.5   eta 0:00:…
   ━━━━━━ 49.5… 18.9   eta 0:00:…
   ━━━━━━ 52.4… 18.6 

In [2]:
%pip install ipywidgets
%pip install --upgrade notebook

  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
Using cached ipywidgets-8.1.7-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl (216 kB)
Using cached widgetsnbextension-4.0.14-py3-none-any.whl (2.2 MB)
   ━━━━━━━━━━━━━ 3/3 [ipywidgets]37m━━━━ 2/3 [ipywidgets]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [28]:
%pip install pydantic-ai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached rich-14.1.0-py3-none-any.whl.metadata (18 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached websockets-15.0.1-cp313-cp313-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached markdown_it_py-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━ 0.0/12.9 ? eta -:--:--
   ━━━━━━ 0.8/… 22.5   eta 0:00:…
   ━━━━━━ 1.0/… 2.2    eta 0:00:…
   ━━╺━━━ 4.7/… 7.7    eta 0:00:…
   ━━━━╸━ 10.0… 12.3   eta 0:00:…
   ━━━━━╸ 12.8… 13.2   eta 0:00:…
   ━━━━━━ 12.9/1… 12.6    0:00:01
          MB      MB/s           
   ━━━━━━━ 0.0/14.1 ? eta -:--:--
   ━━╺━━━ 5.2/… 26.4   eta 0:00:…
   ━━━━╺━ 10.5… 26.2   eta 0:00:…
   ━━━━

In [38]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


Day 1: Ingest and Index Your Data
We need to:
Download and process data from the repo
Put it inside a search engine
Make the search engine available to our agent


In [5]:
# Complete Implementation. Let's now put everything together into a reusable function:
import io
import zipfile
import requests
import frontmatter

def read_repo_data(repo_owner, repo_name):
    """
    Download and parse all markdown files from a GitHub repository.
    
    Args:
        repo_owner: GitHub username or organization
        repo_name: Repository name
    
    Returns:
        List of dictionaries containing file content and metadata
    """
    prefix = 'https://codeload.github.com' 
    url = f'{prefix}/{repo_owner}/{repo_name}/zip/refs/heads/main'
    resp = requests.get(url)
    
    if resp.status_code != 200:
        raise Exception(f"Failed to download repository: {resp.status_code}")

    repository_data = []
    zf = zipfile.ZipFile(io.BytesIO(resp.content))
    
    for file_info in zf.infolist():
        filename = file_info.filename
        filename_lower = filename.lower()

        if not (filename_lower.endswith('.md') 
            or filename_lower.endswith('.mdx')):
            continue
    
        try:
            with zf.open(file_info) as f_in:
                content = f_in.read().decode('utf-8', errors='ignore')
                post = frontmatter.loads(content)
                data = post.to_dict()
                data['filename'] = filename
                repository_data.append(data)
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue
    
    zf.close()
    return repository_data


In [6]:
# We can now use this function for different repositories:

dtc_faq = read_repo_data('DataTalksClub', 'faq')
evidently_docs = read_repo_data('evidentlyai', 'docs')

print(f"FAQ documents: {len(dtc_faq)}")
print(f"Evidently documents: {len(evidently_docs)}")


FAQ documents: 1219
Evidently documents: 95


DAY 2 : Chunking and Intelligent Processing for Data
We will cover multiple ways of chunking data:
Simple character-based chunking
Paragraph and section-based chunking
Intelligent chunking with LLM



In [8]:
# "sliding window" method. This method creates overlapping chunks of text, which can help preserve context across chunk boundaries.

def sliding_window(seq, size, step):
    if size <= 0 or step <= 0:
        raise ValueError("size and step must be positive")

    n = len(seq)
    result = []
    for i in range(0, n, step):
        chunk = seq[i:i+size]
        result.append({'start': i, 'chunk': chunk})
        if i + size >= n:
            break

    return result


In [19]:
# Let's process all the documents:
if 'evidently_docs' not in globals():
    evidently_docs = read_repo_data('evidentlyai', 'docs')

evidently_chunks = []
for doc in evidently_docs:
    doc_copy = doc.copy()
    doc_content = doc_copy.pop('content')
    chunks = sliding_window(doc_content, 2000, 1000)
    for chunk in chunks:
        chunk.update(doc_copy)
    evidently_chunks.extend(chunks)


In [20]:
# Splitting by Paragraphs and Sections

import re
text = evidently_docs[45]['content']
paragraphs = re.split(r"\n\s*\n", text.strip())

# use regex to split by headers
import re

def split_markdown_by_level(text, level=2):
    """
    Split markdown text by a specific header level.
    
    :param text: Markdown text as a string
    :param level: Header level to split on
    :return: List of sections as strings
    """
    # This regex matches markdown headers
    # For level 2, it matches lines starting with "## "
    header_pattern = r'^(#{' + str(level) + r'} )(.+)$'
    pattern = re.compile(header_pattern, re.MULTILINE)

    # Split and keep the headers
    parts = pattern.split(text)
    
    sections = []
    for i in range(1, len(parts), 3):
        # We step by 3 because regex.split() with
        # capturing groups returns:
        # [before_match, group1, group2, after_match, ...]
        # here group1 is "## ", group2 is the header text
        header = parts[i] + parts[i+1]  # "## " + "Title"
        header = header.strip()

        # Get the content after this header
        content = ""
        if i+2 < len(parts):
            content = parts[i+2].strip()

        if content:
            section = f'{header}\n\n{content}'
        else:
            section = header
        sections.append(section)
    
    return sections

# If we want to split by second-level headers, that's what we do:
sections = split_markdown_by_level(text, level=2)

#Now we iterate over all the docs to create the final result:
evidently_chunks = []

for doc in evidently_docs:
    doc_copy = doc.copy()
    doc_content = doc_copy.pop('content')
    sections = split_markdown_by_level(doc_content, level=2)
    for section in sections:
        section_doc = doc_copy.copy()
        section_doc['section'] = section
        evidently_chunks.append(section_doc)



In [6]:
# ...replace the existing OpenAI setup cell with this...

from dotenv import load_dotenv
import os
from openai import OpenAI

# Load variables from .env (if present) into environment
load_dotenv()

# Set your OpenAI API key here if not already set
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError(
        "OPENAI_API_KEY not found. Create a .env or set the env var before running."
    )

# Create client explicitly with the key
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Safe verification (only shows a short prefix)
print("OPENAI_API_KEY present, prefix:", OPENAI_API_KEY[:6] + "..." )

OPENAI_API_KEY present, prefix: sk-pro...


In [7]:
import sys
print(sys.executable)   # should point to /.../.venv/.../python

/mnt/data/Github/7-Day-AI-Agents-Email-Crash-Course/.venv/bin/python


In [8]:
import os
k = os.getenv("OPENAI_API_KEY")
print("Key loaded:", bool(k), "prefix:", (k[:6] + "...") if k else None)

Key loaded: True prefix: sk-pro...


In [9]:
from openai import OpenAI
import os



openai_client = OpenAI()


def llm(prompt, model='gpt-4o-mini'):
    messages = [
        {"role": "user", "content": prompt}
    ]

    response = openai_client.responses.create(
        model='gpt-4o-mini',
        input=messages
    )

    return response.output_text


In [10]:
prompt_template = """
Split the provided document into logical sections
that make sense for a Q&A system.

Each section should be self-contained and cover
a specific topic or concept.

<DOCUMENT>
{document}
</DOCUMENT>

Use this format:

## Section Name

Section content with all relevant details

---

## Another Section Name

Another section content

---
""".strip()


In [11]:
# create a function for intelligent chunking:
def intelligent_chunking(text):
    prompt = prompt_template.format(document=text)
    response = llm(prompt)
    sections = response.split('---')
    sections = [s.strip() for s in sections if s.strip()]
    return sections


In [ ]:
#Now we apply this to every document:
from tqdm.auto import tqdm

evidently_chunks = []

for doc in tqdm(evidently_docs):
    doc_copy = doc.copy()
    doc_content = doc_copy.pop('content')

    sections = intelligent_chunking(doc_content)
    for section in sections:
        section_doc = doc_copy.copy()
        section_doc['section'] = section
        evidently_chunks.append(section_doc)


Day 3: Add Search
In particular, we will:
Build a lexical search for exact matches and keywords
Implement semantic search using embeddings
Combine them with a hybrid search

In [9]:
# prepared the docs
if 'read_repo_data' not in globals():
    raise NameError("Function 'read_repo_data' is not defined. Please run the cell that defines it first.")

evidently_docs = read_repo_data('evidentlyai', 'docs')

evidently_chunks = []

for doc in evidently_docs:
    doc_copy = doc.copy()
    doc_content = doc_copy.pop('content')
    chunks = sliding_window(doc_content, 2000, 1000)
    for chunk in chunks:
        chunk.update(doc_copy)
    evidently_chunks.extend(chunks)


In [11]:
# index this data with minsearch
from minsearch import Index

index = Index(
    text_fields=["chunk", "title", "description", "filename"],
    keyword_fields=[]
)

index.fit(evidently_chunks)

query = 'What should be in a test dataset for AI evaluation?'
results = index.search(query)

In [12]:
from minsearch import Index
from minsearch import Index, VectorSearch

dtc_faq = read_repo_data('DataTalksClub', 'faq')

de_dtc_faq = [d for d in dtc_faq if 'data-engineering' in d['filename']]

faq_index = Index(
    text_fields=["question", "content"],
    keyword_fields=[]
)
faq_index.fit(de_dtc_faq)

query = "Can I join the course now?"  # <-- Add this line
results = faq_index.search(query)
print(results)

[{'id': '3f1424af17', 'question': 'Course: Can I still join the course after the start date?', 'sort_order': 3, 'content': "Yes, even if you don't register, you're still eligible to submit the homework.\n\nBe aware, however, that there will be deadlines for turning in homeworks and the final projects. So don't leave everything for the last minute.", 'filename': 'faq-main/_questions/data-engineering-zoomcamp/general/003_3f1424af17_course-can-i-still-join-the-course-after-the-start.md'}, {'id': '9e508f2212', 'question': 'Course: When does the course start?', 'sort_order': 1, 'content': "The next cohort starts January 13th, 2025. More info at [DTC](https://datatalks.club/blog/guide-to-free-online-courses-at-datatalks-club.html).\n\n- Register before the course starts using this [link](https://airtable.com/shr6oVXeQvSI5HuWD).\n- Join the [course Telegram channel with announcements](https://t.me/dezoomcamp).\n- Don’t forget to register in DataTalks.Club's Slack and join the channel.", 'file

In [5]:
# For vector search, we need to turn our documents into vectors (embeddings).
# We will use the sentence-transformers library for this purpose.
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

dtc_faq = read_repo_data('DataTalksClub', 'faq')
de_dtc_faq = [d for d in dtc_faq if 'data-engineering' in d['filename']]

# Let's take one document from the FAQ dataset:
record = de_dtc_faq[2]
text = record['question'] + ' ' + record['content']
v_doc = embedding_model.encode(text)

# We combine the question and answer text, then convert it to an embedding vector.
# Let's do the same for the query:
query = 'I just found out about the course. Can I enroll now?'
v_query = embedding_model.encode(query)

# This is how we compute similarity between the query and document vectors:
similarity = v_query.dot(v_doc) 





In [17]:
# turn our docs into embeddings
from tqdm.auto import tqdm
import numpy as np

faq_embeddings = []

for d in tqdm(de_dtc_faq):
    text = d['question'] + ' ' + d['content']
    v = embedding_model.encode(text)
    faq_embeddings.append(v)

faq_embeddings = np.array(faq_embeddings)
print(faq_embeddings.shape)  # should be (number_of_docs, embedding_dimension)

100%|██████████| 449/449 [01:39<00:00,  4.50it/s]

(449, 768)


In [18]:
# Now let's use VectorSearch:
from minsearch import VectorSearch

faq_vindex = VectorSearch()
faq_vindex.fit(faq_embeddings, de_dtc_faq)



In [19]:
query = 'Can I join the course now?'
q = embedding_model.encode(query)
results = faq_vindex.search(q)


In [20]:
# We first create an embedding for our query (q), then search for similar document embeddings.
# You can easily do the same with the Evidently docs (but only use the chunk field for embeddings):
evidently_embeddings = []

for d in tqdm(evidently_chunks):
    v = embedding_model.encode(d['chunk'])
    evidently_embeddings.append(v)

evidently_embeddings = np.array(evidently_embeddings)

evidently_vindex = VectorSearch()
evidently_vindex.fit(evidently_embeddings, evidently_chunks)



100%|██████████| 575/575 [05:02<00:00,  1.90it/s]


In [21]:
# Hybrid search - combines both exact keyword matches and vector search.
query = 'Can I join the course now?'

text_results = faq_index.search(query, num_results=5)

q = embedding_model.encode(query)
vector_results = faq_vindex.search(q, num_results=5)

final_results = text_results + vector_results


In [14]:
# Putting this together. But before we can use it in our agent, we need to organize the code. Let's put all the code into different functions:
def text_search(query):
    return faq_index.search(query, num_results=5)

def vector_search(query):
    q = embedding_model.encode(query)
    return faq_vindex.search(q, num_results=5)

def hybrid_search(query):
    text_results = text_search(query)
    vector_results = vector_search(query)
    
    # Combine and deduplicate results
    seen_ids = set()
    combined_results = []

    for result in text_results + vector_results:
        if result['filename'] not in seen_ids:
            seen_ids.add(result['filename'])
            combined_results.append(result)
    
    return combined_results



In [17]:
from minsearch import Index, VectorSearch
from sentence_transformers import SentenceTransformer
import numpy as np

# Make sure de_dtc_faq is defined
dtc_faq = read_repo_data('DataTalksClub', 'faq')
de_dtc_faq = [d for d in dtc_faq if 'data-engineering' in d['filename']]

faq_index = Index(
	text_fields=["question", "content"],
	keyword_fields=[]
)
faq_index.fit(de_dtc_faq)

# Load embedding model if not already loaded
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

# Create embeddings for FAQ documents
faq_embeddings = []
for d in de_dtc_faq:
	text = d['question'] + ' ' + d['content']
	v = embedding_model.encode(text)
	faq_embeddings.append(v)
faq_embeddings = np.array(faq_embeddings)

# Create VectorSearch index
faq_vindex = VectorSearch()
faq_vindex.fit(faq_embeddings, de_dtc_faq)

# Now we can use hybrid_search() to get results from both methods:
query = 'Can I join the course now?'
results = hybrid_search(query)
print(results)

[{'id': '3f1424af17', 'question': 'Course: Can I still join the course after the start date?', 'sort_order': 3, 'content': "Yes, even if you don't register, you're still eligible to submit the homework.\n\nBe aware, however, that there will be deadlines for turning in homeworks and the final projects. So don't leave everything for the last minute.", 'filename': 'faq-main/_questions/data-engineering-zoomcamp/general/003_3f1424af17_course-can-i-still-join-the-course-after-the-start.md'}, {'id': '9e508f2212', 'question': 'Course: When does the course start?', 'sort_order': 1, 'content': "The next cohort starts January 13th, 2025. More info at [DTC](https://datatalks.club/blog/guide-to-free-online-courses-at-datatalks-club.html).\n\n- Register before the course starts using this [link](https://airtable.com/shr6oVXeQvSI5HuWD).\n- Join the [course Telegram channel with announcements](https://t.me/dezoomcamp).\n- Don’t forget to register in DataTalks.Club's Slack and join the channel.", 'file

Day 4: Agents and Tools
In particular, we will:
Learn what makes an AI system "agentic" through tool use
Build an agent that can use the search function
Use Pydantic AI to make it easier to implement agents



In [7]:
# ...replace the existing OpenAI setup cell with this...

from dotenv import load_dotenv
import os
from openai import OpenAI

# Load variables from .env (if present) into environment
load_dotenv()

# Set your OpenAI API key here if not already set
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sk-..."  # <-- Replace with your actual key

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError(
        "OPENAI_API_KEY not found. Create a .env or set the env var before running."
    )

# Create client explicitly with the key
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Safe verification (only shows a short prefix)
print("OPENAI_API_KEY present, prefix:", OPENAI_API_KEY[:6] + "..." )

OPENAI_API_KEY present, prefix: sk-pro...


In [8]:
import sys
print(sys.executable)   # should point to /.../.venv/.../python

/mnt/data/Github/7-Day-AI-Agents-Email-Crash-Course/.venv/bin/python


In [9]:
import os
k = os.getenv("OPENAI_API_KEY")
print("Key loaded:", bool(k), "prefix:", (k[:6] + "...") if k else None)

Key loaded: True prefix: sk-pro...


In [10]:
import os
import openai

# Read API key from environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY not found. Set it in your environment before running.")

openai_client = openai.OpenAI(api_key=OPENAI_API_KEY)

user_prompt = "I just discovered the course, can I join now?"

chat_messages = [
    {"role": "user", "content": user_prompt}
 ]

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
 )

print(response.output_text)

Whether you can join a course that you've just discovered depends on the specific course's policies. Here are a few steps to follow:

1. **Check Enrollment Dates:** Look for information about enrollment deadlines.
2. **Contact the Instructor or Administrator:** Reach out to them directly to ask if late enrollment is possible.
3. **Review Prerequisites:** Ensure you meet any prerequisites or requirements for the course.
4. **Look for Alternatives:** If the course is closed, see if there are similar courses you can join later.

If you let me know more about the course, I might be able to provide specific advice!


In [11]:
# function calling
# describe this function  using a special description format
text_search_tool = {
    "type": "function",
    "name": "text_search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
} 

In [12]:
system_prompt = """
You are a helpful assistant for a course. 
"""

question = "I just discovered the course, can I join now?"

chat_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
]

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=[text_search_tool]
)

print(response.output)

[ResponseFunctionToolCall(arguments='{"query":"join course"}', call_id='call_UHXj7lgAJCwZ4dBWi3Ygjokb', name='text_search', type='function_call', id='fc_0cad09a5cb6f0d4c0068dd9fed161481938511cf338220b18b', status='completed')]


In [27]:
#  invoke the function with these arguments:
import json

if 'response' not in globals():
    raise NameError("Variable 'response' is not defined. Please run the cell that creates 'response' first.")

call = response.output[0]

arguments = json.loads(call.arguments)
result = text_search(**arguments)

call_output = {
    "type": "function_call_output",
    "call_id": call.call_id,
    "output": json.dumps(result),
}

# extending the chat_messages list and sending the entire conversation history back to the LLM
chat_messages.append(call)
chat_messages.append(call_output)

response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=[text_search_tool]
)

print(response.output_text) 

Yes, you can still join the course even after the start date. While it's ideal to register before the course begins, you are allowed to submit homework and participate. Just keep in mind that there are deadlines for submitting assignments and projects, so plan accordingly!

If you're interested, you can register using [this link](https://airtable.com/shr6oVXeQvSI5HuWD) before the next cohort starts on January 13th, 2025. 

Feel free to ask if you have any more questions!


In [6]:
# system prompt
# system prompt is very important: it influences how the agent behaves
system_prompt = """
You are a helpful assistant for a course. 

Use the search tool to find relevant information from the course materials before answering questions.

If you can find specific information through search, use it to provide accurate answers.
If the search doesn't return relevant results, let the user know and provide general guidance.
"""

# if we want the agent to make multiple search queries, we can modify the prompt:
system_prompt = """
You are a helpful assistant for a course. 

Always search for relevant information before answering. 
If the first search doesn't give you enough information, try different search terms.

Make multiple searches if needed to provide comprehensive answers.
"""


In [5]:
# Pydantic AI
# A library to create AI agents using Pydantic models
from typing import List, Any

def text_search(query: str) -> List[Any]:
    """
    Perform a text-based search on the FAQ index.

    Args:
        query (str): The search query string.

    Returns:
        List[Any]: A list of up to 5 search results returned by the FAQ index.
    """
    return faq_index.search(query, num_results=5)

    

In [18]:
# We can now define an agent with Pydantic AI and give it the text_search tool:
from pydantic_ai import Agent

agent = Agent(
    name="faq_agent",
    instructions=system_prompt,
    tools=[text_search],
    model='gpt-4o-mini'
)


In [27]:
# Ensure faq_index is defined before running the agent
from minsearch import Index

de_dtc_faq = [d for d in dtc_faq if 'data-engineering' in d['filename']]

faq_index = Index(
	text_fields=["question", "content"],
	keyword_fields=[]
)
faq_index.fit(de_dtc_faq)

# Now run the agent
question = "I just discovered the course, can I join now?"

result = await agent.run(user_prompt=question)



In [ ]:
# In Jupyter, use 'await' directly for async functions:
result = await agent.run(user_prompt=question)
print(result)


Day 5: Evaluation
In particular, we will cover:
Build a logging system to track agent interactions
Create automated evaluation using AI as a judge
Generate test data automatically
Measure agent performance with metrics


In [15]:
# We can start collecting this data ourselves: start with vibe checking.
from typing import List, Any
from pydantic_ai import Agent


def text_search(query: str) -> List[Any]:
    """
    Perform a text-based search on the FAQ index.

    Args:
        query (str): The search query string.

    Returns:
        List[Any]: A list of up to 5 search results returned by the FAQ index.
    """
    return faq_index.search(query, num_results=5)


system_prompt = """
You are a helpful assistant for a  course. 

Use the search tool to find relevant information from the course materials before answering questions.

If you can find specific information through search, use it to provide accurate answers.
If the search doesn't return relevant results, let the user know and provide general guidance.
"""

from pydantic_ai import Agent

agent = Agent(
    name="faq_agent",
    instructions=system_prompt,
    tools=[text_search],
    model='gpt-4o-mini'
)

question = "how do I install Kafka in Python?"
result = await agent.run(user_prompt=question)

In [17]:
# we'll implement a simple logging system ourselves: we will just write logs to json files.
# In practice, we want to send these logs to some log collection system, or use specialized LLM evaluation tools like Evidently, LangWatch or Arize Phoenix.
# ModelMessagesTypeAdapter.dump_python(messages) to convert internal message format into regular Python dictionaries. This makes it easier to save it to JSON and process later.
# Add source parameter to tracks where the question came from. We start with "user" but later we'll use AI-generated queries.

from pydantic_ai.messages import ModelMessagesTypeAdapter

def log_entry(agent, messages, source="user"):
    tools = []

    for ts in agent.toolsets:
        tools.extend(ts.tools.keys())

    dict_messages = ModelMessagesTypeAdapter.dump_python(messages)

    return {
        "agent_name": agent.name,
        "system_prompt": agent._instructions,
        "provider": agent.model.system,
        "model": agent.model.model_name,
        "tools": tools,
        "messages": dict_messages,
        "source": source
    }


# This code extracts the key information from our agent:
# the configuration (name, prompt, model)
# available tools
# complete message history (user input, tool calls, responses)


In [18]:
# Let's write these logs to a folder:
import json
import secrets
from pathlib import Path
from datetime import datetime


LOG_DIR = Path('logs')
LOG_DIR.mkdir(exist_ok=True)


def serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    raise TypeError(f"Type {type(obj)} not serializable")


def log_interaction_to_file(agent, messages, source='user'):
    entry = log_entry(agent, messages, source)

    ts = entry['messages'][-1]['timestamp']
    ts_str = ts.strftime("%Y%m%d_%H%M%S")
    rand_hex = secrets.token_hex(3)

    filename = f"{agent.name}_{ts_str}_{rand_hex}.json"
    filepath = LOG_DIR / filename

    with filepath.open("w", encoding="utf-8") as f_out:
        json.dump(entry, f_out, indent=2, default=serializer)

    return filepath

# This code:
# Creates a logs directory (if not created previously)
# Generates unique filenames with timestamp and random hex
# Saves complete interaction logs as JSON files
# Handles datetime serialization (using the serialized function)


In [19]:
# Now we can interact with it and do some vibe checking:

question = input()
result = await agent.run(user_prompt=question)
print(result.output)
log_interaction_to_file(agent, result.new_messages())

# This creates a simple interactive loop where:
# User enters a question
# Agent processes it and responds
# Complete interaction is logged to a file


 what do I need to do for the certificate?


To earn your certificate for the course, you need to follow these guidelines:

1. **Complete the Peer-Reviewed Capstone Projects**: You must finish the capstone projects on time. Completing the homework is not mandatory, especially if you join late.

2. **Participation in a Live Cohort**: You can only receive a certificate if you finish the course with a live cohort. Certificates are not awarded for self-paced participation.

3. **Check Your Profile**: There will be announcements in the course Telegram channel regarding:
   - Ensuring that your full name is correctly displayed on the certificate. You can edit your profile on the Course Management webpage.
   - Notifications about when the grading is completed.

4. **Access Your Certificate**: Once grading is done, you will need to follow the instructions provided in the course channels to generate your certificate. You can find your certificate in your course profile after following the announcements.

Feel free to reach out if you nee

PosixPath('logs/faq_agent_20251001_221330_330b16.json')

In [20]:
# Adding References
# Let's fix it by adjusting the prompt:
system_prompt = """
You are a helpful assistant for a course.  

Use the search tool to find relevant information from the course materials before answering questions.  

If you can find specific information through search, use it to provide accurate answers.

Always include references by citing the filename of the source material you used.  
When citing the reference, replace "faq-main" by the full path to the GitHub repository: "https://github.com/DataTalksClub/faq/blob/main/"
Format: [LINK TITLE](FULL_GITHUB_LINK)

If the search doesn't return relevant results, let the user know and provide general guidance.  
""".strip()

# Create another version of agent, let's call it faq_agent_v2
agent = Agent(
    name="faq_agent_v2",
    instructions=system_prompt,
    tools=[text_search],
    model='gpt-4o-mini'
)


In [21]:
# LLM as a Judge
# The idea is simple: we use one LLM to evaluate the outputs of another LLM. This works because LLMs are good at following detailed evaluation criteria.
# Our system prompt for the judge (we'll call it "evaluation agent" because it sounds cooler) can look like that:

evaluation_prompt = """
Use this checklist to evaluate the quality of an AI agent's answer (<ANSWER>) to a user question (<QUESTION>).
We also include the entire log (<LOG>) for analysis.

For each item, check if the condition is met. 

Checklist:

- instructions_follow: The agent followed the user's instructions (in <INSTRUCTIONS>)
- instructions_avoid: The agent avoided doing things it was told not to do  
- answer_relevant: The response directly addresses the user's question  
- answer_clear: The answer is clear and correct  
- answer_citations: The response includes proper citations or sources when required  
- completeness: The response is complete and covers all key aspects of the request
- tool_call_search: Is the search tool invoked? 

Output true/false for each check and provide a short explanation for your judgment.
""".strip()


In [22]:
# Define a Pydantic class with the expected response structure, and the LLM will produce output that matches this schema exactly.
from pydantic import BaseModel

class EvaluationCheck(BaseModel):
    check_name: str
    justification: str
    check_pass: bool

class EvaluationChecklist(BaseModel):
    checklist: list[EvaluationCheck]
    summary: str

# This code defines the structure we expect from our evaluation:
# Each check has a name, justification, and pass/fail result
# The overall evaluation includes a list of checks and a summary


In [23]:
# With Pydantic AI in order to make the output follow the specified class, we use the parameter output_type:

eval_agent = Agent(
    name='eval_agent',
    model='gpt-5-nano',
    instructions=evaluation_prompt,
    output_type=EvaluationChecklist
)


In [27]:
import os
print(os.listdir('./logs'))

['faq_agent_20251001_221330_330b16.json']


In [28]:
# Evaluate different model 
# We use XML markup because it's easier and more clear for LLMs to understand the input. 
# XML tags help the model see the structure and boundaries of different sections in the prompt.
user_prompt_format = """
<INSTRUCTIONS>{instructions}</INSTRUCTIONS>
<QUESTION>{question}</QUESTION>
<ANSWER>{answer}</ANSWER>
<LOG>{log}</LOG>
""".strip()

#Let's fill it in. First, define a helper function for loading JSON log files:
def load_log_file(log_file):
    with open(log_file, 'r') as f_in:
        log_data = json.load(f_in)
        log_data['log_file'] = log_file
        return log_data

# We also add the filename in the result - it'll help us with tracking later.
# Now let's use it:
log_record = load_log_file('./logs/faq_agent_20251001_221330_330b16.json')

instructions = log_record['system_prompt']
question = log_record['messages'][0]['parts'][0]['content']
answer = log_record['messages'][-1]['parts'][0]['content']
log = json.dumps(log_record['messages'])

user_prompt = user_prompt_format.format(
    instructions=instructions,
    question=question,
    answer=answer,
    log=log
)

#The user input is ready and we can test it!
result = await eval_agent.run(user_prompt, output_type=EvaluationChecklist)

checklist = result.output
print(checklist.summary)

for check in checklist.checklist:
    print(check)

# This code:
# Loads a saved interaction log
# Extracts the key components (instructions, question, answer, full log)
# Formats them into the evaluation prompt
# Runs the evaluation agent
# Prints the results


All checklist items pass. The assistant used the course material search results to provide an accurate and complete answer about certificate requirements, and will include citations from the sources.
check_name='instructions_follow' justification='The assistant will search the course materials before answering, per <INSTRUCTIONS> and then answer.' check_pass=True
check_name='instructions_avoid' justification='No disallowed content; the assistant will use approved tools.' check_pass=True
check_name='answer_relevant' justification='The answer directly explains what is needed for the certificate.' check_pass=True
check_name='answer_clear' justification='Provides enumerated steps and clear requirements.' check_pass=True
check_name='answer_citations' justification='Citations can be included referencing course FAQ items; tool provided data includes file names; will cite accordingly.' check_pass=True
check_name='completeness' justification='Covers capstone, live cohort, profile/name, grading 

In [6]:
# Note that we're putting the entire conversation log into the prompt, which is not really necessary. We can reduce it to make it less verbose.
# For example, like that:
def simplify_log_messages(messages):
    log_simplified = []

    for m in messages:
        parts = []
    
        for original_part in m['parts']:
            part = original_part.copy()
            kind = part['part_kind']
    
            if kind == 'user-prompt':
                del part['timestamp']
            if kind == 'tool-call':
                del part['tool_call_id']
            if kind == 'tool-return':
                del part['tool_call_id']
                del part['metadata']
                del part['timestamp']
                # Replace actual search results with placeholder to save tokens
                part['content'] = 'RETURN_RESULTS_REDACTED'
            if kind == 'text':
                del part['id']
    
            parts.append(part)
    
        message = {
            'kind': m['kind'],
            'parts': parts
        }
    
        log_simplified.append(message)
    return log_simplified

# We make it simpler:
# remove timestamps and IDs that aren't needed for evaluation
# replace actual search results with a placeholder
# keep only the essential structure

# This is helpful because it reduces the number of tokens we send to the evaluation model, which lowers the costs and speeds up evaluation


In [30]:
# Data Generation
# Ask an LLM to generate a question based on the record
# We use this question as input to our agent and log the answers.
question_generation_prompt = """
You are helping to create test questions for an AI agent that answers questions about a data engineering course.

Based on the provided FAQ content, generate realistic questions that students might ask.

The questions should:

- Be natural and varied in style
- Range from simple to complex
- Include both specific technical questions and general course questions

Generate one question for each record.
""".strip()

class QuestionsList(BaseModel):
    questions: list[str]

question_generator = Agent(
    name="question_generator",
    instructions=question_generation_prompt,
    model='gpt-4o-mini',
    output_type=QuestionsList
)


In [31]:
#  let's sample 10 records from our dataset using Python's built-in random.sample function:
import random

sample = random.sample(de_dtc_faq, 10)
prompt_docs = [d['content'] for d in sample]
prompt = json.dumps(prompt_docs)

result = await question_generator.run(prompt)
questions = result.output.questions


In [32]:
# Now we simply iterate over each of the question, ask our agent and log the results:
from tqdm.auto import tqdm

for q in tqdm(questions):
    print(q)

    result = await agent.run(user_prompt=q)
    print(result.output)

    log_interaction_to_file(
        agent,
        result.new_messages(),
        source='ai-generated'
    )

    print()

# Using AI for generating test data is quite powerful. It can help us get data faster and sometimes cover edge cases we won't think about.
# There are limitations too:
# AI-generated questions might not reflect real user behavior
# It may miss important edge cases that only real users encounter
# They may not capture the full complexity of real user queries


  0%|          | 0/10 [00:00<?, ?it/s]

How can I improve the audio quality of the lectures in this course?
To improve the audio quality of the lectures in the course, you can try the following methods:

1. **Download and Playback**: Download the videos and use **VLC media player**. This player allows you to set the audio volume to **200%** of the original, which can help in enhancing the sound quality.

2. **Use Auto-generated Captions**: You can enable the **auto-generated captions** directly on YouTube as a way to improve clarity and understanding of the content.

These tips can help mitigate issues with low audio levels in the lectures you've encountered. 

Reference: [Kafka: Python videos have low audio and are hard to follow up](https://github.com/DataTalksClub/faq/blob/main/faq-main/_questions/data-engineering-zoomcamp/module-6/010_348f696873_kafka-python-videos-have-low-audio-and-are-hard-to.md)

What should I check if my PYTHONPATH isn't set correctly for PySpark?
If your `PYTHONPATH` isn't set correctly for PySpark

In [9]:
from pathlib import Path
LOG_DIR = Path('./logs')

import json

def load_log_file(log_file):
    with open(log_file, 'r', encoding='utf-8') as f:
        return json.load(f)

In [10]:
# The logs are ready, so we can run evaluation on them with our evaluation agent
# First, collect all the AI-generated logs for the v2 agent:
eval_set = []

for log_file in LOG_DIR.glob('*.json'):
    if 'faq_agent_v2' not in log_file.name:
        continue

    log_record = load_log_file(log_file)
    if log_record['source'] != 'ai-generated':
        continue

    eval_set.append(log_record)


In [35]:
# Define the evaluate_log_record function
async def evaluate_log_record(agent, log_record):
    """
    Evaluates a single log record using the provided agent.

    Args:
        agent: The evaluation agent (should have an async run method).
        log_record: The log record to evaluate.

    Returns:
        The evaluation result from the agent.
    """
    result = await agent.run(user_prompt=log_record)
    return result

In [13]:
from tqdm.auto import tqdm

In [14]:
# Define the evaluation function
async def evaluate_log_record(agent, log_record):
    """
    Run the evaluation agent on a single log record.

    Args:
        agent: The evaluation agent (should have an async run method).
        log_record: The log record to evaluate.

    Returns:
        The evaluation result from the agent.
    """
    return await agent.run(user_prompt=log_record)

In [16]:
from pydantic_ai import Agent

# Example instructions for the evaluation agent
eval_instructions = """
You are an evaluation agent. Your job is to review AI-generated logs and assess their quality based on a checklist.
"""

# Define the evaluation agent (customize tools and model as needed)
eval_agent = Agent(
    name="eval_agent",
    instructions=eval_instructions,
    tools=[],  # Add any tools your evaluation agent should use
    model='gpt-4o-mini'
)

In [17]:
# And evaluate them:
eval_results = []

for log_record in tqdm(eval_set):
    eval_result = await evaluate_log_record(eval_agent, log_record)
    eval_results.append((log_record, eval_result))

# This code:
# Loops through each AI-generated log
# Runs our evaluation agent on it
# Stores both the original log and evaluation result


  0%|          | 0/10 [00:00<?, ?it/s]

In [18]:
# Print available attributes for debugging
print(dir(eval_result))
print(eval_result)

# Example extraction if checks are in a dict
if hasattr(eval_result, "checks"):
    checks = eval_result.checks
    # If checks is a dict of {name: pass/fail}
    if isinstance(checks, dict):
        row.update(checks)
    # If checks is a list of objects
    elif isinstance(checks, list):
        for c in checks:
            row[c.check_name] = c.check_pass
else:
    print("No checks attribute found in eval_result.")

['__annotations__', '__class__', '__class_getitem__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__replace__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_new_message_index', '_output_tool_name', '_set_output_tool_return', '_state', '_traceparent', '_traceparent_value', 'all_messages', 'all_messages_json', 'new_messages', 'new_messages_json', 'output', 'timestamp', 'usage']
AgentRunResult(output="It seems like you've provided a string of keywords or a fragment of information related to AI-generated logs. To assist you better, I would need more context or specific log details that you 

In [21]:
for log_record in tqdm(eval_set):
    eval_result = await evaluate_log_record(eval_agent, log_record)
    # Create a new row dictionary for each record
    row = {}
    row['log_record'] = log_record

    import json
    try:
        output_data = json.loads(eval_result.output)
    except Exception:
        output_data = eval_result.output

    if isinstance(output_data, dict) and 'checklist' in output_data:
        checks = {c['check_name']: c['check_pass'] for c in output_data['checklist']}
        row.update(checks)
    else:
        row['output'] = output_data

    eval_results.append(row)

  0%|          | 0/10 [00:00<?, ?it/s]

In [22]:
import json

# Inspect the output first
print(eval_result.output)

# Try to parse if it's a JSON string
try:
    output_data = json.loads(eval_result.output)
except Exception:
    output_data = eval_result.output

# If output_data is a dict and contains 'checklist', extract checks
if isinstance(output_data, dict) and 'checklist' in output_data:
    checks = {c['check_name']: c['check_pass'] for c in output_data['checklist']}
    row.update(checks)
else:
    # Fallback: just store the output as a string
    row['output'] = output_data

It looks like you're providing a structure or headers, but I need more context or specific logs to evaluate. Please provide the AI-generated logs you want me to assess based on the quality checklist.


In [25]:
import json
from tqdm.auto import tqdm

eval_results = []

for log_record in tqdm(eval_set):
    eval_result = await evaluate_log_record(eval_agent, log_record)
    row = {}
    row['log_record'] = log_record

    try:
        output_data = json.loads(eval_result.output)
    except Exception:
        output_data = eval_result.output

    if isinstance(output_data, dict) and 'checklist' in output_data:
        checks = {c['check_name']: c['check_pass'] for c in output_data['checklist']}
        row.update(checks)
    else:
        row['output'] = output_data

    eval_results.append(row)

  0%|          | 0/10 [00:00<?, ?it/s]

In [29]:
import json

rows = []

for log_record, eval_result in eval_results:
    # If log_record is a string, try to parse it
    if isinstance(log_record, str):
        try:
            log_record = json.loads(log_record)
        except Exception:
            pass  # If not JSON, leave as is

    # Now you can safely access its fields if it's a dict
    if isinstance(log_record, dict) and 'messages' in log_record:
        messages = log_record['messages']
        row = {
            'file': log_record.get('log_file', {}).get('name', ''),
            'question': messages[0]['parts'][0]['content'],
            'answer': messages[-1]['parts'][0]['content'],
        }
        # Extract checks if available
        if hasattr(eval_result.output, 'checklist'):
            checks = {c.check_name: c.check_pass for c in eval_result.output.checklist}
            row.update(checks)
        rows.append(row)
    else:
        # Fallback: store raw log_record and eval_result
        rows.append({'log_record': log_record, 'eval_result': eval_result})


In [30]:
print(log_record)

log_record


In [34]:
# Now each row is a simple key-value dictionary, so we can create a DataFrame:
import pandas as pd
df = pd.DataFrame(eval_results)
df.head()


,log_record,output
0,"{'agent_name': 'faq_agent_v2', 'system_prompt'...",It seems like you've provided a series of term...
1,"{'agent_name': 'faq_agent_v2', 'system_prompt'...",It seems like you've entered a string of terms...
2,"{'agent_name': 'faq_agent_v2', 'system_prompt'...",It seems like you've entered a series of terms...
3,"{'agent_name': 'faq_agent_v2', 'system_prompt'...",It seems like your input is incomplete or uncl...
4,"{'agent_name': 'faq_agent_v2', 'system_prompt'...",It seems like your message might be incomplete...


In [36]:
import pandas as pd
df_evals = pd.DataFrame(rows)

# overall stats
df_evals.mean(numeric_only=True)

Series([], dtype: float64)

In [37]:
# Evaluating functions and tools
# We also have the search function, which we can evaluate using standard information retrieval metrics. For example:
# Precision and Recall: How many relevant results were retrieved vs. how many relevant results were missed
# Hit Rate: Percentage of queries that return at least one relevant result
# MRR (Mean Reciprocal Rank): Reflects the position of the first relevant result in the ranking

def evaluate_search_quality(search_function, test_queries):
    results = []
    
    for query, expected_docs in test_queries:
        search_results = search_function(query, num_results=5)
        
        # Calculate hit rate
        relevant_found = any(doc['filename'] in expected_docs for doc in search_results)
        
        # Calculate MRR
        for i, doc in enumerate(search_results):
            if doc['filename'] in expected_docs:
                mrr = 1 / (i + 1)
                break
        else:
            mrr = 0
            
        results.append({
            'query': query,
            'hit': relevant_found,
            'mrr': mrr
        })
    return results

